In [43]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import requests
from bs4 import BeautifulSoup
import pandas as pd
import string
import re

#Initializing in order to scrape website as agent not as crawler
headers = {
    'User-Agent': 'My User Agent 1.0'
}

#Start with first page
page = 1

#Initializing
final_title_list = []
body_list = []
date_list = []
title_list = None
p_list = []

#Generic ULR of website
link = "https://100percentfedup.com/"

#Loop to get approx 1000 news articles according to length of the body list
while len(body_list) <= 1000: 
    
    #Modified URL in order to navigate to next page
    add = "page/"+str(page)+"/"
    
    #Generic Url to get fake news
    url = link + "category/politics/" + add
    
    print(url)
    response = requests.get(url, headers=headers)
    if response.status_code==200:  
        soup = BeautifulSoup(response.content, 'html.parser')  

        #Scrapping All the titles first
        titles = []
        title_list = soup.select('div.wpdev-post-grid-item h2')
        for idx,t in enumerate(title_list):
            titles.append(t.text)

        #Scrapping date and body of fake news using previously scrapped titles
        for t in titles:
            
            #Some preprocsessig in order to get the URL of pertiicular article
            tokens = re.split(r'\W+', t.lower().replace(r"…“”’‘",""))
            click = "-".join(tokens)
            new_url = link + click + "/"
            
            #Hitting every title's URL in order to extract content (body) and date of the article
            response = requests.get(new_url, headers=headers)
            if response.status_code==200:  
                soup = BeautifulSoup(response.content, 'html.parser') 

                #Extracting the date
                date = []
                date = soup.select("h4")                   

                #Extracting the body paragraphs
                body = ""
                p_list = soup.select("div.entry-content p")
                
                #Scraping only text hence p_list[1:] since first element is not simple body of text
                for p in p_list[1:]:
                    if len(p.text) > 0:
                        body = body + p.text
                                                  
                #Appending to global list only when body is found
                if(len(body) > 0):
                    body_list.append(body)
                    final_title_list.append(t)
                    date_list.append(date[0].get_text())
                    
    print(len(body_list),len(final_title_list),len(date_list))
    
    #Going to next page
    page = page + 1

https://100percentfedup.com/category/politics/page/1/
17 17 17
https://100percentfedup.com/category/politics/page/2/
37 37 37
https://100percentfedup.com/category/politics/page/3/
49 49 49
https://100percentfedup.com/category/politics/page/4/
63 63 63
https://100percentfedup.com/category/politics/page/5/
77 77 77
https://100percentfedup.com/category/politics/page/6/
93 93 93
https://100percentfedup.com/category/politics/page/7/
105 105 105
https://100percentfedup.com/category/politics/page/8/
121 121 121
https://100percentfedup.com/category/politics/page/9/
133 133 133
https://100percentfedup.com/category/politics/page/10/
150 150 150
https://100percentfedup.com/category/politics/page/11/
166 166 166
https://100percentfedup.com/category/politics/page/12/
186 186 186
https://100percentfedup.com/category/politics/page/13/
211 211 211
https://100percentfedup.com/category/politics/page/14/
235 235 235
https://100percentfedup.com/category/politics/page/15/
256 256 256
https://100percentfedu

In [36]:
#Creating 2 dataset from the scrapped data 
i = 0
data_title = []
data_body = []
while i < len(body_list):
    data_title.append((date_list[i],final_title_list[i],'fake'))
    data_body.append((date_list[i],body_list[i],'fake'))
    i = i + 1
df_only_statement = pd.DataFrame(data_title, columns=['date', 'statement', 'label'])
df_only_body = pd.DataFrame(data_body, columns=['date', 'body', 'label'])

#to get csv
df_only_statement.to_csv('1000_fake_only_statement.csv')
df_only_body.to_csv('1000_fake_only_body.csv')